# Cross-Match Catalogs using NWAY

## Import Libraries

In [1]:
from astropy.io import fits
import numpy as np
from astropy.table import Table

## Import Catalog Data

In [2]:
observatory = '/home/mfonseca/fastdisk2/data/'
my_computer = '/home/polaris/Lab_Astro/data/'
server = '/workspace/data/'

directory = my_computer

In [3]:
# EMU catalogs
emu_catalog_path = directory + 'survey_data/EMU_data/EMU_0102-32/EMU_0102-32_1comp.fits'
emu_patched_catalog_path = directory + 'survey_data/EMU_data/EMU_0102-32/EMU_0102-32_1comp_withoutpatches.fits'
emuXviking_catalog_path = ''

# DES catalogs
desy6gold_catalog_path = directory + 'survey_data/DES_data/DESY6GOLD_in_EMU_0102-32_clean.fits'

# VIKING catalogs
viking_catalog_path = directory + "survey_data/VIKING_data/VIKINGDR5_in_EMU_0102-32_clean.fits"

# CatWISE catalogs
catwise_catalog_path = directory + 'survey_data/CATWISE_data/CATWISE2020_in_EMU_0102-32_clean.fits'

## Prepare the catalogs for NWAY input
The NWAY library has multiple requirements for a catalog to serve as input.
1. The fits data table needs to have a extension name. It is used as a prefix for the columns copied to the output catalog, so each column from each catalog is called "PREFIXOFCATALOG_NAMEOFCOLUMN"
2. The fits header of the catalogs needs to have the SKYAREA keyword. This keyword has the area on the sky in square degrees covered by the catalog. Since I don't have the documentation for EMU data, I must calculate the area using the mosaic image.
3. The position error needs to be a column with a single value, called positional error, so we need to obtain it from the right ascension and declination errors.

### Change extension name
This process is also possible using just the NWAY library. Check page 8 of the NWAY documentation
Changes the extension name of each catalog, it becomes the prefix of the columns from that catalog after running nway

In [4]:
def change_hdu_name(fits_file, hdu_index, new_name):
    with fits.open(fits_file, mode='update') as hdul:
        # Change the name of the specified HDU
        hdul[hdu_index].header['EXTNAME'] = new_name
        # Save changes to the FITS file
        hdul.flush()

In [5]:
change_hdu_name(emu_catalog_path, hdu_index=1, new_name="EMU")
change_hdu_name(emu_patched_catalog_path, hdu_index=1, new_name="EMU")

change_hdu_name(desy6gold_catalog_path, hdu_index=1, new_name='DESY6')
change_hdu_name(viking_catalog_path, hdu_index=1, new_name="VKG")
change_hdu_name(catwise_catalog_path, hdu_index=1, new_name="CAT")

### Create and assign value to SKYAREA keyword

Create the SKYAREA in the FITS header and assign a value to it.

In [6]:
def create_skyarea_keyword(fits_file, skyarea_value):
    """
    Create the SKYAREA keyword in the FITS header and assign a value to it.

    Parameters:
    fits_file (str): Path to the FITS file.
    skyarea_value (float): Value to assign to the SKYAREA keyword (in square degrees).
    """
    with fits.open(fits_file, mode='update') as hdul:
        header = hdul[1].header
        
        header['SKYAREA'] = skyarea_value
        
        hdul.flush()
        print(f"SKYAREA keyword set to {skyarea_value} in {fits_file}.")

In [7]:
def calculate_skyarea(ra_interval, dec_interval):
    """
    Calculate the area of the sky.

    Parameters:
    ra_interval (list): List of interval of RA in degrees
    dec_interval (list): List of interval of DEC in degrees

    """
    delta_ra_rad  = np.radians(abs(ra_interval[1]-ra_interval[0]))
    delta_dec_rad = np.sin(np.radians(dec_interval[1])) - np.sin(np.radians(dec_interval[0]))

    solid_angle_sr = delta_ra_rad*delta_dec_rad
    solid_angle_deg2 = solid_angle_sr * (180 / np.pi) ** 2

    return solid_angle_deg2

In [8]:
patch1 = calculate_skyarea([16.77, 18.32], [-30.84, -30.12])
patch2 = calculate_skyarea([14.95, 16.49], [-35.16, -34.96])
patch3 = calculate_skyarea([12.54, 13.97], [-32.79, -32.05])

total_patch = patch1 + patch2 + patch3

print(f"Patch 1 Sky Area: {patch1} deg^2")
print(f"Patch 2 Sky Area: {patch2} deg^2")
print(f"Patch 3 Sky Area: {patch3} deg^2")
print(f"Total Patch Area: {total_patch} deg^2")

Patch 1 Sky Area: 0.9617694716176918 deg^2
Patch 2 Sky Area: 0.2521135638049071 deg^2
Patch 3 Sky Area: 0.8932636233122696 deg^2
Total Patch Area: 2.1071466587348686 deg^2


In [9]:
# Acoording to documentation it is indeed around 30 degrees
area_emu_survey = calculate_skyarea([12.0,19.0], [-35.16,-30.00])
print(area_emu_survey)

area_emu_patched = area_emu_survey - total_patch
print(area_emu_patched)

30.425887052030664
28.318740393295794


In [10]:
# The limits are derived from the limit imposed when querying the catalogs
area_des_survey = calculate_skyarea([11.8,19.1], [-35.2,-29.9])


area_viking_survey = calculate_skyarea([11.8,19.1], [-35.2,-29.9])
area_catwise_survey = calculate_skyarea([11.8,19.1], [-35.2,-29.9])

# EMU area
create_skyarea_keyword(emu_catalog_path, area_emu_survey)

# DES area
create_skyarea_keyword(desy6gold_catalog_path, area_des_survey-1.54)

# VIKING area
create_skyarea_keyword(viking_catalog_path, area_viking_survey-2.1071466587348686)

# CatWISE area
create_skyarea_keyword(catwise_catalog_path, area_catwise_survey)

SKYAREA keyword set to 30.425887052030664 in /home/polaris/Lab_Astro/data/survey_data/EMU_data/EMU_0102-32/EMU_0102-32_1comp.fits.
SKYAREA keyword set to 31.061035431358185 in /home/polaris/Lab_Astro/data/survey_data/DES_data/DESY6GOLD_in_EMU_0102-32_clean.fits.
SKYAREA keyword set to 30.493888772623315 in /home/polaris/Lab_Astro/data/survey_data/VIKING_data/VIKINGDR5_in_EMU_0102-32_clean.fits.
SKYAREA keyword set to 32.601035431358184 in /home/polaris/Lab_Astro/data/survey_data/CATWISE_data/CATWISE2020_in_EMU_0102-32_clean.fits.


## Fix ID Column in EMU survey

In [11]:

def create_id_column_from_source_name(fits_file):
    with fits.open(fits_file, mode='update') as hdul:
        data = hdul[1].data

        if 'Source_Name' not in data.columns.names:
            raise ValueError("The FITS file does not contain a 'Source_Name' column.")
        
        if 'ID' in data.columns.names:
            raise ValueError("The FITS file already contains an 'ID' column.")

        source_names = data['Source_Name']

        id_data = [name.split('EMUJ')[-1] if 'EMUJ' in name else '' for name in source_names]

        id_col = fits.Column(name='ID', format='20A', array=id_data)  # '20A' for a string column of max length 20

        new_columns = hdul[1].columns + fits.ColDefs([id_col])
        new_hdu = fits.BinTableHDU.from_columns(new_columns)

        hdul[1] = new_hdu

        hdul.flush()
        print("Created new 'ID' column with the number part of 'Source_Name'.")

# Creates a command for NWAY

## Without Magnitude

In [12]:
def build_nway_command_nomag(output_path, input_catalogs, radius=5, nway_path='/root/miniconda3/envs/cigale/bin/nway.py'):
    '''
    Build a nway.py crossmatch command string for magnitudes.

    Args:
    output_path (str): Path to the output file.
    input_catalogs (list of tuples): List of (file_path, pos_error) tuples.
    radius (float, optional): Matching radius (default 5 arcsec).
    nway_path (str, optional): Path to nway.py script.

    Returns:
    str: Full nway.py command string.
    '''

    cmd = f'{nway_path}'

    # Add catalogs and positional errors
    for file_path, pos_error in input_catalogs:
        cmd += f" {file_path} {pos_error}"

    # Output file and radius
    cmd += f" --out={output_path} --radius {radius}"

    print(cmd)
   
    return cmd

In [13]:

input_catalogs = [
    ('/workspace/data/survey_data/VIKINGDR5_in_EMU_0102-32.fits', 0.1),
    ('/workspace/data/survey_data/DESY6GOLD_in_EMU_0102-32.fits', 0.1),
    ('/data/mfonseca/survey_data/CATWISE2020_in_EMU_0102-32.fits', 0.2)
]

output_path = '/workspace/data/cross_match/EMU_0103-32_DESY6_VKG_CAT_noMags_PSF.fits'

# Build and print the command
build_nway_command_nomag(output_path, input_catalogs)

/root/miniconda3/envs/cigale/bin/nway.py /workspace/data/survey_data/VIKINGDR5_in_EMU_0102-32.fits 0.1 /workspace/data/survey_data/DESY6GOLD_in_EMU_0102-32.fits 0.1 /data/mfonseca/survey_data/CATWISE2020_in_EMU_0102-32.fits 0.2 --out=/workspace/data/cross_match/EMU_0103-32_DESY6_VKG_CAT_noMags_PSF.fits --radius 5


'/root/miniconda3/envs/cigale/bin/nway.py /workspace/data/survey_data/VIKINGDR5_in_EMU_0102-32.fits 0.1 /workspace/data/survey_data/DESY6GOLD_in_EMU_0102-32.fits 0.1 /data/mfonseca/survey_data/CATWISE2020_in_EMU_0102-32.fits 0.2 --out=/workspace/data/cross_match/EMU_0103-32_DESY6_VKG_CAT_noMags_PSF.fits --radius 5'

## With Magnitude

In [14]:
def build_nway_command_mag(output_path, input_catalogs, mags_dict, radius=5, nway_path='~/.local/bin/nway.py'):
    '''
    Build a nway.py crossmatch command string for magnitudes.

    Args:
        output_path (str): Path to the output file.
        input_catalogs (list of tuples): List of (file_path, pos_error) tuples.
        mags_dict (dict): Dictionary where keys are prefixes (DES, VKG, CAT) and values are lists of magnitudes.
        radius (float, optional): Matching radius (default 5 arcsec).
        nway_path (str, optional): Path to nway.py script.

    Returns:
        str: Full nway.py command string.
    '''
    cmd = f"{nway_path}"

    # Add catalogs and positional errors
    for file_path, pos_error in input_catalogs:
        cmd += f" {file_path} {pos_error}"

    # Output file and radius
    cmd += f" --out={output_path} --radius {radius}"

    # Magnitudes
    for prefix, mags in mags_dict.items():
        for mag in mags:
            cmd += f" --mag {prefix}:{mag} auto"

    print(cmd)
    return cmd

In [15]:
input_catalogs = [
    ('/data/mfonseca/survey_data/DES_data/DESY6GOLD_in_EMU_0102-32_magauto.fits', 0.1),
    ('/data/mfonseca/survey_data/VIKING_data/VIKINGDR5_in_EMU_0102-32_allapermag3.fits', 0.1),
    ('/data/mfonseca/survey_data/CATWISE_data/CATWISE2020_in_EMU_0102-32.fits', 0.2)
]

output_path = '/home/polaris/Lab_Astro/data/cross_match/EMU_0103-32_DESY6_VKG_CAT_Mags_PSF/EMU_0103-32_DESY6_VKG_CAT_Mags_PSF.fits'

mags_dict = {
    'DESY6': ['mag_auto_g_extcorr', 'mag_auto_r_extcorr', 'mag_auto_i_extcorr', 'mag_auto_z_extcorr', 'mag_auto_y_extcorr'],
    'VKG': ['mag_petro_y_ab_extcorr', 'mag_petro_j_ab_extcorr', 'mag_petro_h_ab_extcorr', 'mag_petro_ks_ab_extcorr'],
    'CAT': ['w1mpro_ab_extcorr', 'w2mpro_ab_extcorr']
}

# Build and print the command
build_nway_command_mag(output_path, input_catalogs, mags_dict)

~/.local/bin/nway.py /data/mfonseca/survey_data/DES_data/DESY6GOLD_in_EMU_0102-32_magauto.fits 0.1 /data/mfonseca/survey_data/VIKING_data/VIKINGDR5_in_EMU_0102-32_allapermag3.fits 0.1 /data/mfonseca/survey_data/CATWISE_data/CATWISE2020_in_EMU_0102-32.fits 0.2 --out=/home/polaris/Lab_Astro/data/cross_match/EMU_0103-32_DESY6_VKG_CAT_Mags_PSF/EMU_0103-32_DESY6_VKG_CAT_Mags_PSF.fits --radius 5 --mag DESY6:mag_auto_g_extcorr auto --mag DESY6:mag_auto_r_extcorr auto --mag DESY6:mag_auto_i_extcorr auto --mag DESY6:mag_auto_z_extcorr auto --mag DESY6:mag_auto_y_extcorr auto --mag VKG:mag_petro_y_ab_extcorr auto --mag VKG:mag_petro_j_ab_extcorr auto --mag VKG:mag_petro_h_ab_extcorr auto --mag VKG:mag_petro_ks_ab_extcorr auto --mag CAT:w1mpro_ab_extcorr auto --mag CAT:w2mpro_ab_extcorr auto


'~/.local/bin/nway.py /data/mfonseca/survey_data/DES_data/DESY6GOLD_in_EMU_0102-32_magauto.fits 0.1 /data/mfonseca/survey_data/VIKING_data/VIKINGDR5_in_EMU_0102-32_allapermag3.fits 0.1 /data/mfonseca/survey_data/CATWISE_data/CATWISE2020_in_EMU_0102-32.fits 0.2 --out=/home/polaris/Lab_Astro/data/cross_match/EMU_0103-32_DESY6_VKG_CAT_Mags_PSF/EMU_0103-32_DESY6_VKG_CAT_Mags_PSF.fits --radius 5 --mag DESY6:mag_auto_g_extcorr auto --mag DESY6:mag_auto_r_extcorr auto --mag DESY6:mag_auto_i_extcorr auto --mag DESY6:mag_auto_z_extcorr auto --mag DESY6:mag_auto_y_extcorr auto --mag VKG:mag_petro_y_ab_extcorr auto --mag VKG:mag_petro_j_ab_extcorr auto --mag VKG:mag_petro_h_ab_extcorr auto --mag VKG:mag_petro_ks_ab_extcorr auto --mag CAT:w1mpro_ab_extcorr auto --mag CAT:w2mpro_ab_extcorr auto'